In [1]:
!pip install pathlib


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pathlib import Path


metrics_output_dir = Path('./res/outputs/adjudication_metrics')

## OCR Metrics

In [3]:
from pathlib import Path
import pandas as pd
import json
ocr_output_dir = Path('./res/outputs/ocr')

#we construct the global list of claims that this process will use
def did_complete_ocr(claim) -> bool:
   #ocr is complete if the claim["documents"] list is not-empty. 
   return len(claim["documents"]) > 0

#read through every json file in the ocr output directory and build a dataframe of the policy id, claim id, and number of documents in the claim
claims_batch_df = pd.DataFrame()
for file in ocr_output_dir.glob("*.json"):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        number_of_documents_in_claim = len(data['documents'])
        did_claim_complete_ocr = did_complete_ocr(data)
        new_row = pd.DataFrame({'policy_id': [data['policy_id']], 'claim_id': [data['claim_id']], 'number_of_documents_in_claim': [number_of_documents_in_claim], 'did_claim_complete_ocr': [did_claim_complete_ocr]})

        claims_batch_df = pd.concat([claims_batch_df, new_row], ignore_index=True)


In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import logging

# --- Assuming claims_batch_df is already loaded from the previous cell ---
# Ensure join key columns in claims_batch_df are strings, just to be safe
if 'policy_id' in claims_batch_df.columns:
    claims_batch_df['policy_id'] = claims_batch_df['policy_id'].astype(str)
if 'claim_id' in claims_batch_df.columns:
    claims_batch_df['claim_id'] = claims_batch_df['claim_id'].astype(str)

print("Data types for join keys in claims_batch_df:")
if 'policy_id' in claims_batch_df.columns and 'claim_id' in claims_batch_df.columns:
    print(claims_batch_df[['policy_id', 'claim_id']].dtypes)
else:
    print("policy_id or claim_id not in claims_batch_df at this point.")


num_claims_in_batch = len(claims_batch_df)
num_claims_in_batch_that_did_complete_ocr = len(claims_batch_df[claims_batch_df['did_claim_complete_ocr'] == True])
num_claims_in_batch_that_did_not_complete_ocr = len(claims_batch_df[claims_batch_df['did_claim_complete_ocr'] == False])

adjudication_output_dir = Path('./res/outputs/adjudication')
ground_truth_file = Path('./res/outputs/ground_truth_claims_decisions/policy_claims_decisions.csv')

#Load the ground truth data frame.
gt_df = pd.read_csv(ground_truth_file, dtype={'policy_number': str, 'claim_number': str})
gt_df['decision'] = gt_df['decision'].astype(str).str.upper()
gt_df['decision'] = gt_df['decision'].replace({'PAID': 'PAY', 'DENIED': 'DENY'})
gt_df = gt_df.rename(columns={'policy_number': 'policy_id', 'claim_number': 'claim_id', 'decision': 'gt_decision', 'status': 'gt_status', 'status_reason': 'gt_status_reason','in_data_set': 'gt_in_data_set'})

print("\nUnique values in 'gt_decision' (original from CSV after replace):", gt_df['gt_decision'].unique())
print(f"Number of DENY records in gt_df before drop_duplicates: {len(gt_df[gt_df['gt_decision'] == 'DENY'])}")

# Drop duplicates from gt_df based on the join keys before merging
gt_df_deduplicated = gt_df.drop_duplicates(subset=['policy_id', 'claim_id'], keep='first')

print(f"Number of DENY records in gt_df after drop_duplicates: {len(gt_df_deduplicated[gt_df_deduplicated['gt_decision'] == 'DENY'])}")



Data types for join keys in claims_batch_df:
policy_id    object
claim_id     object
dtype: object

Unique values in 'gt_decision' (original from CSV after replace): ['PAY' 'DENY']
Number of DENY records in gt_df before drop_duplicates: 62
Number of DENY records in gt_df after drop_duplicates: 53


In [5]:
# --- New Diagnostic Section ---
# Get the set of (policy_id, claim_id) tuples from claims_batch_df
claims_batch_keys = set(zip(claims_batch_df['policy_id'], claims_batch_df['claim_id']))

# Get the (policy_id, claim_id) pairs that are "DENY" in the deduplicated gt_df
gt_deny_keys_df = gt_df_deduplicated[gt_df_deduplicated['gt_decision'] == 'DENY'][['policy_id', 'claim_id']]
gt_deny_keys = set(zip(gt_deny_keys_df['policy_id'], gt_deny_keys_df['claim_id']))

print(f"\nNumber of unique (policy_id, claim_id) keys in claims_batch_df: {len(claims_batch_keys)}")
print(f"Number of unique (policy_id, claim_id) keys for DENY in gt_df_deduplicated: {len(gt_deny_keys)}")

# Find which "DENY" keys from gt_df are present in claims_batch_df
matching_deny_keys = claims_batch_keys.intersection(gt_deny_keys)
print(f"Number of DENY keys from gt_df that are also found in claims_batch_df: {len(matching_deny_keys)}")

if len(matching_deny_keys) > 0:
    print("Sample of matching DENY keys found in both DataFrames:")
    print(list(matching_deny_keys)[:5])
else:
    print("No DENY keys from gt_df were found in claims_batch_df.")
    if gt_deny_keys: # Corrected line: check if the set is not empty
        print("Sample of DENY keys from gt_df (that were not found in claims_batch_df):")
        print(list(gt_deny_keys)[:5])
# --- End New Diagnostic Section ---


#Now we want to perform a left outer join of the claims_batch_df with the gt_df on the policy_id and claim_id fields.
claims_batch_df = pd.merge(
    claims_batch_df,
    gt_df_deduplicated[['policy_id', 'claim_id', 'gt_decision', 'gt_status', 'gt_status_reason']], # use deduplicated gt_df
    on=['policy_id', 'claim_id'],
    how='left'
)

missing_gt_decision_df = claims_batch_df[claims_batch_df['gt_decision'].isnull()]
print(f"\nNumber of rows in merged claims_batch_df with missing gt_decision: {len(missing_gt_decision_df)}")

print("\nUnique values in 'gt_decision' in the merged claims_batch_df:")
print(claims_batch_df['gt_decision'].unique())
print("Value counts for 'gt_decision' in the merged claims_batch_df:")
print(claims_batch_df['gt_decision'].value_counts(dropna=False))



Number of unique (policy_id, claim_id) keys in claims_batch_df: 94
Number of unique (policy_id, claim_id) keys for DENY in gt_df_deduplicated: 53
Number of DENY keys from gt_df that are also found in claims_batch_df: 33
Sample of matching DENY keys found in both DataFrames:
[('4151524576', '202410100052'), ('4151516802', '202406170099'), ('4151582003', '202502060010'), ('4151525980', '202411010051'), ('4151484579', '202503280007')]

Number of rows in merged claims_batch_df with missing gt_decision: 0

Unique values in 'gt_decision' in the merged claims_batch_df:
['PAY' 'DENY']
Value counts for 'gt_decision' in the merged claims_batch_df:
gt_decision
PAY     61
DENY    33
Name: count, dtype: int64


In [6]:
#Let's not filter out any rows from the claims_batch_df that did NOT complete OCR.

claims_batch_df_with_ocr_results = claims_batch_df[claims_batch_df['did_claim_complete_ocr'] == True]



In [7]:
#Go through each JSON file in the adjudication_output_dir and extract the claim_id and policy_id from 
#json data. Then use the claim and policy ids to add the json data to the claims_batch_df
adjudication_data_df = pd.DataFrame()
for file in adjudication_output_dir.glob("*.json"):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        claim_id = data['claim_id']
        policy_id = data['policy_id']
        adjudication_decision = (data['decision']['status']).upper()
       
        new_row = pd.DataFrame({'policy_id': [policy_id], 'claim_id': [claim_id], 'adjudication_decision': [adjudication_decision], 'adjudication_data': [data]})
        adjudication_data_df = pd.concat([adjudication_data_df, new_row], ignore_index=True)

#Now we have a claims_batch_df with the adjudication data added to it.
#Take the claims_batch_df_with_ocr_results and left outer join it with the adjudication_data_df on the policy_id and claim_id fields.
claims_batch_df_with_ocr_results = pd.merge(claims_batch_df_with_ocr_results, adjudication_data_df, on=['policy_id', 'claim_id'], how='left')

num_claims_marked_refer = len(claims_batch_df_with_ocr_results[claims_batch_df_with_ocr_results['adjudication_decision'] == 'REFER'])
num_claims_marked_pay = len(claims_batch_df_with_ocr_results[claims_batch_df_with_ocr_results['adjudication_decision'] == 'PAY'])
num_claims_marked_deny = len(claims_batch_df_with_ocr_results[claims_batch_df_with_ocr_results['adjudication_decision'] == 'DENY'])

#We need to filter all the rows where the adjudication_devision is "REFER"
claims_batch_df_with_ocr_results = claims_batch_df_with_ocr_results[claims_batch_df_with_ocr_results['adjudication_decision'] != 'REFER']












In [8]:
# In[9]
# Generate summary statistics for the claims_batch_df_with_ocr_results dataframe.
# Treat the column adjudication_decision to be the predicted class and the column gt_decision to be the ground truth class.

# You might need to install scikit-learn if you haven't already
# !pip install scikit-learn

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import json
from datetime import datetime
from pathlib import Path # Ensure Path is imported

true_column = 'gt_decision'
pred_column = 'adjudication_decision'

print(f"--- Initial Data Overview for '{true_column}' and '{pred_column}' ---")
if true_column in claims_batch_df_with_ocr_results:
    print(f"\nValue counts for '{true_column}':")
    print(claims_batch_df_with_ocr_results[true_column].value_counts(dropna=False))
else:
    print(f"\nError: True column '{true_column}' not found in the DataFrame.")
    # exit() # Or handle error appropriately

if pred_column in claims_batch_df_with_ocr_results:
    print(f"\nValue counts for '{pred_column}':")
    print(claims_batch_df_with_ocr_results[pred_column].value_counts(dropna=False))
else:
    print(f"\nError: Predicted column '{pred_column}' not found in the DataFrame.")
    # exit() # Or handle error appropriately

print("\n--- Preparing data for metrics calculation ---")
# Create a copy for metrics calculation
metrics_df = claims_batch_df_with_ocr_results[[true_column, pred_column]].copy()

# Drop rows where ground truth is missing, as they cannot be used for evaluation
initial_rows = len(metrics_df)
metrics_df.dropna(subset=[true_column], inplace=True)
rows_after_gt_dropna = len(metrics_df)
removed_rows_count = initial_rows - rows_after_gt_dropna
print(f"Removed {removed_rows_count} rows due to missing '{true_column}'.")

if metrics_df.empty:
    print("No data remaining after removing missing ground truth values. Cannot calculate metrics.")
else:
    y_true = metrics_df[true_column]
    y_pred = metrics_df[pred_column].copy() # Use .copy() to avoid SettingWithCopyWarning on fillna

    # Handle potential NaNs in the prediction column for rows where ground truth is known.
    y_pred_original_nans = y_pred.isnull().sum()
    if y_pred_original_nans > 0:
        print(f"Found {y_pred_original_nans} NaN values in '{pred_column}' for rows with valid '{true_column}'. Replacing with 'NO_PREDICTION'.")
        y_pred.fillna("NO_PREDICTION", inplace=True)
    else:
        print(f"No NaN values found in '{pred_column}' for rows with valid '{true_column}'.")

    print("\n--- Calculating Metrics ---")

    # Determine all unique labels present in true and predicted values
    labels = sorted(list(set(y_true) | set(y_pred)))
    print(f"Unique labels considered for metrics: {labels}")
    
    if not labels:
        print("No labels found to calculate metrics.")
    else:
        # Accuracy
        accuracy = accuracy_score(y_true, y_pred)
        
        # Precision per class
        precision_per_class_values = precision_score(y_true, y_pred, labels=labels, average=None, zero_division=0)
        precision_per_class_dict = {label: score for label, score in zip(labels, precision_per_class_values)}

        # Recall per class
        recall_per_class_values = recall_score(y_true, y_pred, labels=labels, average=None, zero_division=0)
        recall_per_class_dict = {label: score for label, score in zip(labels, recall_per_class_values)}
        
        # Classification Report (provides precision, recall, F1-score, and support)
        report_str = classification_report(y_true, y_pred, labels=labels, zero_division=0)
        
        # Calculate actual counts from y_true (for data used in metrics)
        actual_pay_in_metrics_data = (y_true == 'PAY').sum()
        actual_deny_in_metrics_data = (y_true == 'DENY').sum()

        # --- Prepare for file output ---
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = f"adjudication_summary_statistics_{timestamp}"
        
        if isinstance(metrics_output_dir, str):
            metrics_output_dir = Path(metrics_output_dir)
            
        json_filename = metrics_output_dir / f"{base_filename}.json"
        md_filename = metrics_output_dir / f"{base_filename}.md"

        # --- JSON Output ---
        metrics_summary_json = {
            "generation_timestamp": datetime.now().isoformat(),
            "data_pipeline_summary": {
                "total_claims_in_initial_batch": num_claims_in_batch,
                "claims_completed_ocr": num_claims_in_batch_that_did_complete_ocr,
                "claims_predicted_pay_by_adjudication_before_refer_filter": num_claims_marked_pay, 
                "claims_predicted_deny_by_adjudication_before_refer_filter": num_claims_marked_deny,
                "claims_predicted_refer_by_adjudication": num_claims_marked_refer 
            },
            "metrics_calculation_summary": {
                "true_column": true_column,
                "predicted_column": pred_column,
                "data_points_used_for_metrics": len(y_true),
                "actual_pay_in_metrics_data": int(actual_pay_in_metrics_data),
                "actual_deny_in_metrics_data": int(actual_deny_in_metrics_data),
                "rows_removed_due_to_missing_ground_truth": removed_rows_count,
                "predicted_nans_replaced_with_NO_PREDICTION": int(y_pred_original_nans),
                "unique_labels_in_metrics": labels
            },
            "overall_accuracy": accuracy,
            "precision_per_class": precision_per_class_dict,
            "recall_per_class": recall_per_class_dict,
            "classification_report_text": report_str
        }
        
        json_filename.parent.mkdir(parents=True, exist_ok=True)

        with open(json_filename, 'w') as f_json:
            json.dump(metrics_summary_json, f_json, indent=4)
        print(f"\nMetrics summary saved to JSON: {json_filename}")

        # --- Markdown Output ---
        md_content = []
        md_content.append(f"# Adjudication Summary Statistics\n")
        md_content.append(f"**Generated at:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        md_content.append(f"## Data Pipeline & Adjudication Counts")
        md_content.append(f"*   Total claims in initial batch (from OCR output): **{num_claims_in_batch}**")
        md_content.append(f"*   Claims that completed OCR: **{num_claims_in_batch_that_did_complete_ocr}**")
        md_content.append(f"*   Claims predicted as PAY by adjudication (before 'REFER' filter for metrics): **{num_claims_marked_pay}**")
        md_content.append(f"*   Claims predicted as DENY by adjudication (before 'REFER' filter for metrics): **{num_claims_marked_deny}**")
        md_content.append(f"*   Claims predicted as REFER by adjudication: **{num_claims_marked_refer}**\n")

        md_content.append(f"## Metrics Calculation Overview")
        md_content.append(f"*   Metrics calculated on **{len(y_true)}** data points (from OCR-completed, non-referred, and having ground truth).")
        md_content.append(f"    *   Actual PAY decisions in this metrics set: **{actual_pay_in_metrics_data}**")
        md_content.append(f"    *   Actual DENY decisions in this metrics set: **{actual_deny_in_metrics_data}**")
        md_content.append(f"*   {removed_rows_count} rows were removed due to missing '{true_column}'.")
        if y_pred_original_nans > 0:
            md_content.append(f"*   **{y_pred_original_nans}** NaN values in '{pred_column}' (for rows with valid '{true_column}') were treated as 'NO_PREDICTION'.")
        else:
            md_content.append(f"*   No NaN values in '{pred_column}' (for rows with valid '{true_column}') needed replacement.")
        md_content.append(f"*   Unique labels considered for metrics: `{labels}`\n")

        md_content.append(f"## Overall Accuracy")
        md_content.append(f"Accuracy: {accuracy:.4f}\n")

        md_content.append(f"## Precision per Class")
        for label_item, score_item in precision_per_class_dict.items(): # Renamed to avoid conflict
            md_content.append(f"*   {label_item}: {score_item:.4f}")
        md_content.append("\n")

        md_content.append(f"## Recall per Class")
        for label_item, score_item in recall_per_class_dict.items(): # Renamed to avoid conflict
            md_content.append(f"*   {label_item}: {score_item:.4f}")
        md_content.append("\n")
        
        md_content.append(f"## Classification Report")
        md_content.append(f"```")
        md_content.append(report_str)
        md_content.append(f"```\n")

        # --- New Section for Detailed Claims Data Table ---
        md_content.append(f"## Detailed Claims Data (Metrics Set)\n")
        md_content.append(f"The following table shows the specific claims that were included in the metrics calculation (after filtering for OCR completion, non-'REFER' adjudication, and presence of a ground truth decision).\n")

        columns_to_dump = [
            'policy_id', 
            'claim_id', 
            'number_of_documents_in_claim', 
            'did_claim_complete_ocr', 
            'gt_decision', 
            'gt_status_reason',
            'adjudication_decision', 
            'adjudication_data'
        ]
        
        # Create the DataFrame for the table based on the indices of y_true
        # This ensures it only contains rows used in the metrics.
        df_for_markdown_table_source = claims_batch_df_with_ocr_results.loc[y_true.index].copy()
        
        # Override 'adjudication_decision' with y_pred (which has NaNs filled)
        df_for_markdown_table_source['adjudication_decision'] = y_pred

        # Ensure all columns are present, add as NA if missing, and select in specified order
        temp_table_data = {}
        for col_name in columns_to_dump:
            if col_name in df_for_markdown_table_source.columns:
                temp_table_data[col_name] = df_for_markdown_table_source[col_name]
            else:
                # If a column is genuinely missing and expected, this will add it with NAs.
                # For 'adjudication_decision', it's already handled by y_pred assignment.
                print(f"Warning: Column '{col_name}' not found for detailed table, will be empty.")
                temp_table_data[col_name] = pd.Series([pd.NA] * len(df_for_markdown_table_source), index=df_for_markdown_table_source.index)
        
        df_for_markdown_table = pd.DataFrame(temp_table_data, columns=columns_to_dump)


        def format_json_cell(data):
            if pd.isna(data):
                return ""
            try:
                # Compact JSON string
                json_str = json.dumps(data, separators=(',', ':'))
                # Escape backticks within the JSON string itself, then wrap with backticks
                json_str_escaped = json_str.replace('`', '\\`')
                return f"`{json_str_escaped}`"
            except TypeError:
                return "`Error: Not JSON serializable`"
            except Exception as e: # Catch any other unexpected error during formatting
                return f"`Error formatting JSON: {str(e)}`"

        if 'adjudication_data' in df_for_markdown_table.columns:
            df_for_markdown_table['adjudication_data'] = df_for_markdown_table['adjudication_data'].apply(format_json_cell)

        # Fill remaining NaN in other columns with empty string for cleaner Markdown table
        df_for_markdown_table = df_for_markdown_table.fillna('') 

        if not df_for_markdown_table.empty:
            markdown_table_string = df_for_markdown_table.to_markdown(index=False)
            md_content.append(markdown_table_string)
        else:
            md_content.append("No data available to display in the detailed claims table (metrics dataset was empty or table generation failed).")
        md_content.append("\n")
        # --- End of New Section ---
        
        md_content.append(f"## Interpretation Notes")
        md_content.append(f"*   Metrics are calculated on {len(y_true)} data points after removing rows with missing '{true_column}'.")
        if y_pred_original_nans > 0:
            md_content.append(f"*   If '{pred_column}' had NaN values, they were treated as a 'NO_PREDICTION' category.")
        md_content.append(f"*   'Support' in the classification report refers to the number of actual instances of each class in '{true_column}'.")
        md_content.append(f"*   If a class has 0 support (no true instances), its recall and F1-score will be 0.")
        md_content.append(f"*   If a class was never predicted, its precision might be 0 (depending on true instances).")
        if 'DENY' not in y_true.unique() and 'DENY' in labels: # Check y_true.unique() instead of labels for this warning
             md_content.append(f"\n**WARNING:** The ground truth data used for these metrics ('{true_column}') does not appear to contain 'DENY' labels among the {len(y_true)} records used.")
             md_content.append(f"This means recall for 'DENY' will be 0, and precision for 'DENY' will also be 0 if 'DENY' was predicted for any non-DENY true case.")
        
        with open(md_filename, 'w', encoding='utf-8') as f_md: # Added encoding
            f_md.write("\n".join(md_content))
        print(f"Metrics summary saved to Markdown: {md_filename}")

        # Display the original print outputs as well for immediate notebook feedback
        print(f"\nAccuracy: {accuracy:.4f}")
        print("\nPrecision per class:")
        for label_item, score_item in precision_per_class_dict.items(): # Renamed to avoid conflict
            print(f"  {label_item}: {score_item:.4f}")
        print("\nRecall per class:")
        for label_item, score_item in recall_per_class_dict.items(): # Renamed to avoid conflict
            print(f"  {label_item}: {score_item:.4f}")
        print("\nClassification Report:")
        print(report_str)
        
        print("\n--- Interpretation Notes (also in Markdown file) ---")
        print(f"* Metrics are calculated on {len(y_true)} data points after removing rows with missing '{true_column}'.")
        if y_pred_original_nans > 0:
            print(f"* If '{pred_column}' had NaN values, they were treated as a 'NO_PREDICTION' category.")
        print(f"* 'Support' in the classification report refers to the number of actual instances of each class in '{true_column}'.")
        print("* If a class has 0 support (no true instances), its recall and F1-score will be 0.")
        print("* If a class was never predicted, its precision might be 0 (depending on true instances).")
        if 'DENY' not in y_true.unique() and 'DENY' in labels: # Check y_true.unique()
             print(f"\nWARNING: The ground truth data used for these metrics ('{true_column}') does not appear to contain 'DENY' labels among the {len(y_true)} records used.")
             print("This means recall for 'DENY' will be 0, and precision for 'DENY' will also be 0 if 'DENY' was predicted for any non-DENY true case.")

--- Initial Data Overview for 'gt_decision' and 'adjudication_decision' ---

Value counts for 'gt_decision':
gt_decision
PAY     57
DENY    27
Name: count, dtype: int64

Value counts for 'adjudication_decision':
adjudication_decision
PAY     76
DENY     8
Name: count, dtype: int64

--- Preparing data for metrics calculation ---
Removed 0 rows due to missing 'gt_decision'.
No NaN values found in 'adjudication_decision' for rows with valid 'gt_decision'.

--- Calculating Metrics ---
Unique labels considered for metrics: ['DENY', 'PAY']

Metrics summary saved to JSON: res/outputs/adjudication_metrics/adjudication_summary_statistics_20250518_163523.json
Metrics summary saved to Markdown: res/outputs/adjudication_metrics/adjudication_summary_statistics_20250518_163523.md

Accuracy: 0.7738

Precision per class:
  DENY: 1.0000
  PAY: 0.7500

Recall per class:
  DENY: 0.2963
  PAY: 1.0000

Classification Report:
              precision    recall  f1-score   support

        DENY       1.00    